In [2]:
import scanpy as sc
import pandas as pd

In [3]:
adata = sc.read_h5ad("/Users/aumchampaneri/Complement-OUD/Multi-Omics Study/data/raw/snrna/GSE225158_BU_OUD_Striatum_refined_all_SeuratObj_N22.h5ad")

In [14]:
# List all obs in adata and print
obs_list = adata.obs.columns.tolist()
# Create a DataFrame with obs names and their type
obs_df = pd.DataFrame({
    'obs_name': obs_list,
    'type': [adata.obs[name].dtype.name for name in obs_list]
})
obs_df

,obs_name,type
0,nCount_RNA,float64
1,nFeature_RNA,int32
2,nCount_SCT,float64
3,nFeature_SCT,int32
4,orig.ident,category
5,scds.hybrid_score,float64
6,scds.keep,category
7,percent.mt,float64
8,miQC.probability,float64
9,miQC.keep,category


In [5]:
# Extract raw count data
# Check what layers are available
print("Available layers:", adata.layers.keys())
print("Raw data in .raw:", adata.raw is not None)

# Option 1: If raw counts are stored in .raw
if adata.raw is not None:
    raw_adata = adata.raw.to_adata()
    print("Extracted raw data shape:", raw_adata.shape)
else:
    # Option 2: If raw counts are in a layer (common layer names)
    if 'counts' in adata.layers:
        raw_adata = sc.AnnData(X=adata.layers['counts'], 
                               obs=adata.obs.copy(), 
                               var=adata.var.copy())
    elif 'raw' in adata.layers:
        raw_adata = sc.AnnData(X=adata.layers['raw'], 
                               obs=adata.obs.copy(), 
                               var=adata.var.copy())
    else:
        # Option 3: Use current X if it's already raw counts
        raw_adata = adata.copy()
        print("Using current X matrix as raw counts")

# Check if the data looks like raw counts (should be integers)
print("Data type:", raw_adata.X.dtype)
print("Min value:", raw_adata.X.min())
print("Max value:", raw_adata.X.max())
print("Sum of first cell:", raw_adata.X[0].sum())

Available layers: KeysView(Layers with keys: )
Raw data in .raw: True
Extracted raw data shape: (98848, 31611)
Data type: float64
Min value: 0.0
Max value: 12005.0
Sum of first cell: 6472.0


In [6]:
# Start with the raw data we extracted
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 🔬 1. PREPROCESSING & QUALITY CONTROL (QC)
print("=== STEP 1: PREPROCESSING & QC ===")

# A. Data already loaded (raw_adata from previous extraction)
print(f"Starting with {raw_adata.n_obs} cells and {raw_adata.n_vars} genes")

# B. Initial QC Metrics
print("\n--- Calculating QC Metrics ---")

# Gene annotations
raw_adata.var['mt'] = raw_adata.var_names.str.startswith('MT-')
raw_adata.var['ribo'] = raw_adata.var_names.str.startswith(('RPS', 'RPL'))
raw_adata.var['hb'] = raw_adata.var_names.str.contains('^HB[^(P)]')  # Hemoglobin genes

# Calculate comprehensive QC metrics
sc.pp.calculate_qc_metrics(raw_adata, percent_top=None, log1p=False, inplace=True)

# Add percentage metrics
raw_adata.obs['pct_counts_mt'] = (raw_adata.obs['n_counts_mt'] / raw_adata.obs['total_counts']) * 100
raw_adata.obs['pct_counts_ribo'] = (raw_adata.obs['n_counts_ribo'] / raw_adata.obs['total_counts']) * 100

# Print QC summary
print(f"Cells: {raw_adata.n_obs}")
print(f"Genes: {raw_adata.n_vars}")
print(f"MT genes: {raw_adata.var['mt'].sum()}")
print(f"Ribosomal genes: {raw_adata.var['ribo'].sum()}")
print(f"Median genes/cell: {raw_adata.obs['n_genes_by_counts'].median():.0f}")
print(f"Median UMI/cell: {raw_adata.obs['total_counts'].median():.0f}")
print(f"Median MT%: {raw_adata.obs['pct_counts_mt'].median():.2f}%")

=== STEP 1: PREPROCESSING & QC ===
Starting with 98848 cells and 31611 genes

--- Calculating QC Metrics ---


KeyError: 'n_counts_mt'

In [15]:
adata

AnnData object with n_obs × n_vars = 98848 × 31393
    obs: 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'orig.ident', 'scds.hybrid_score', 'scds.keep', 'percent.mt', 'miQC.probability', 'miQC.keep', 'dropletQC.nucFrac', 'dropletQC.keep', 'integrated_snn_res.1', 'seurat_clusters', 'X', 'ID', 'Region', 'Pair', 'Case', 'Sex', 'Race', 'Age', 'BMI', 'PMI', 'pH', 'RIN', 'Tissue.Storage.Time.mo.b', 'Dx_OUD', 'Dx_Substances', 'Dx_Comorbid', 'Dur.OUD', 'DSM.IV.SUD', 'DSM.IV.Psych', 'Blood.Toxicology', 'Infxn.Dx', 'Medications.ATODc', 'Tobacco.ATOD', 'Manner.of.Death', 'Cause.of.Death', 'Index.27', 'Index.28', 'i7.index.seq', 'i5.index.seq', 'DSM.IV.OUD', 'DSM.IV.AUD', 'DSM.IV.CUD', 'integrated_snn_res.0.5', 'celltype1', 'celltype2', 'integrated_snn_res.2', 'celltype3', 'integrated_snn_res.0.1', 'level1', 'level2', 'n_genes'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
    obsm: 'X_umap'

In [3]:
adata2 = sc.read_h5ad("/Users/aumchampaneri/Complement-OUD/Multi-Omics Study/data/processed/snrna_qc/GSE225158_reprocessed_pearson.h5ad")
adata2


AnnData object with n_obs × n_vars = 75942 × 30527
    obs: 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'orig.ident', 'scds.hybrid_score', 'scds.keep', 'percent.mt', 'miQC.probability', 'miQC.keep', 'dropletQC.nucFrac', 'dropletQC.keep', 'integrated_snn_res.1', 'seurat_clusters', 'X', 'ID', 'Region', 'Pair', 'Case', 'Sex', 'Race', 'Age', 'BMI', 'PMI', 'pH', 'RIN', 'Tissue.Storage.Time.mo.b', 'Dx_OUD', 'Dx_Substances', 'Dx_Comorbid', 'Dur.OUD', 'DSM.IV.SUD', 'DSM.IV.Psych', 'Blood.Toxicology', 'Infxn.Dx', 'Medications.ATODc', 'Tobacco.ATOD', 'Manner.of.Death', 'Cause.of.Death', 'Index.27', 'Index.28', 'i7.index.seq', 'i5.index.seq', 'DSM.IV.OUD', 'DSM.IV.AUD', 'DSM.IV.CUD', 'integrated_snn_res.0.5', 'celltype1', 'celltype2', 'integrated_snn_res.2', 'celltype3', 'integrated_snn_res.0.1', 'level1', 'level2', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'leiden_res_0.1', 'leiden_res_0.3', 'leiden

In [5]:
adata2.var

,features,mt,ribo,hb,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,n_cells,means,variances,residual_variances,highly_variable_rank,highly_variable_nbatches,highly_variable_intersection,highly_variable
0,MIR1302-2HG,False,False,False,97,0.000991,99.901870,98.0,18,0.000237,0.000237,0.438924,NaN,0,False,False
1,OR4F5,False,False,False,61,0.000617,99.938289,61.0,23,0.000303,0.000303,0.435800,NaN,0,False,False
2,AL627309.1,False,False,False,4893,0.054761,95.049976,5413.0,2265,0.031076,0.032718,1.079290,NaN,0,False,False
3,AL627309.2,False,False,False,26,0.000263,99.973697,26.0,20,0.000263,0.000263,0.338362,NaN,0,False,False
4,AL627309.5,False,False,False,4043,0.046324,95.909882,4579.0,2491,0.038174,0.052308,1.319018,2494.0,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31589,AC091182.2,False,False,False,3,0.000030,99.996965,3.0,3,0.000040,0.000040,0.061625,NaN,0,False,False
31595,AC126175.2,False,False,False,3,0.000030,99.996965,3.0,3,0.000040,0.000040,0.030030,NaN,0,False,False
31597,AL355922.1,False,False,False,6,0.000081,99.993930,8.0,6,0.000105,0.000158,0.069767,NaN,0,False,False
31601,AC016383.3,False,False,False,4,0.000040,99.995953,4.0,3,0.000040,0.000040,0.052058,NaN,0,False,False


In [ ]:
# Explore the features column which contains gene names
print("Gene names are stored in the 'features' column:")
print(f"Number of genes: {len(adata2.var['features'])}")
print(f"First 10 gene names:")
print(adata2.var['features'].head(10).tolist())

# Check if gene names are also the index
print(f"\nAre gene names also the var index? {(adata2.var.index == adata2.var['features']).all()}")

# Show some gene statistics
print(f"\nGene name statistics:")
print(f"Unique gene names: {adata2.var['features'].nunique()}")
print(f"Total entries: {len(adata2.var['features'])}")
print(f"Any duplicates? {adata2.var['features'].duplicated().any()}")

# Look at some specific gene types
mt_genes = adata2.var[adata2.var['mt']]['features'].tolist()
ribo_genes = adata2.var[adata2.var['ribo']]['features'].tolist()

print(f"\nMitochondrial genes found: {len(mt_genes)}")
print(f"First 5 MT genes: {mt_genes[:5]}")
print(f"\nRibosomal genes found: {len(ribo_genes)}")
print(f"First 5 ribosomal genes: {ribo_genes[:5]}")

In [1]:
# Check regional distribution
print('Regional breakdown:')
print(adata.obs['Region'].value_counts())

print('\nRegion by Condition:')
crosstab = pd.crosstab(adata.obs['Region'], adata.obs['Dx_OUD'])
print(crosstab)

print('\nRegion by Sex:')
crosstab_sex = pd.crosstab(adata.obs['Region'], adata.obs['Sex'])
print(crosstab_sex)

Regional breakdown:


NameError: name 'adata' is not defined